In [1]:
import numpy as np
import torch
from grnewt import NewtonSummary, compute_Hg, nesterov_lrs

In [2]:
"""
P = 10
num_expes = 1000

dct_errors = {'torch eigh': 0.,
              'torch eig': 0.,
              'numpy eigh': 0.,
              'numpy eig': 0.,
              'torch eigh (float64)': 0.}

val = -1e-6
for i in range(num_expes):
    Hm = torch.randn(P, P)
    Hp = Hm.t() @ Hm
    Hd, Hu = torch.linalg.eigh(Hp)
    Hd[0] = val
    H = Hu @ Hd.diag() @ Hu.t()

    dct_errors['torch eigh'] += abs(torch.linalg.eigh(H).eigenvalues[0].item() - val) / num_expes
    dct_errors['torch eig'] += abs(torch.linalg.eig(H).eigenvalues.real.sort().values[0].item() - val) / num_expes
    dct_errors['numpy eigh'] += abs(np.linalg.eigh(H.numpy()).eigenvalues[0] - val) / num_expes
    dct_errors['numpy eig'] += abs(sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0] - val) / num_expes
    dct_errors['torch eigh (float64)'] += abs(torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item() - val) / num_expes

for k, v in dct_errors.items():
    print(k, ':', v)
"""

"\nP = 10\nnum_expes = 1000\n\ndct_errors = {'torch eigh': 0.,\n              'torch eig': 0.,\n              'numpy eigh': 0.,\n              'numpy eig': 0.,\n              'torch eigh (float64)': 0.}\n\nval = -1e-6\nfor i in range(num_expes):\n    Hm = torch.randn(P, P)\n    Hp = Hm.t() @ Hm\n    Hd, Hu = torch.linalg.eigh(Hp)\n    Hd[0] = val\n    H = Hu @ Hd.diag() @ Hu.t()\n\n    dct_errors['torch eigh'] += abs(torch.linalg.eigh(H).eigenvalues[0].item() - val) / num_expes\n    dct_errors['torch eig'] += abs(torch.linalg.eig(H).eigenvalues.real.sort().values[0].item() - val) / num_expes\n    dct_errors['numpy eigh'] += abs(np.linalg.eigh(H.numpy()).eigenvalues[0] - val) / num_expes\n    dct_errors['numpy eig'] += abs(sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0] - val) / num_expes\n    dct_errors['torch eigh (float64)'] += abs(torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item() - val) / num_expes\n\nfor k, v in dct_errors.items():\n    print(k, ':', v)\n"

In [3]:
S = 10

g = torch.randn(S)

Hm = torch.randn(S, S)
Hp = Hm.t() @ Hm
Hd, Hu = torch.linalg.eigh(Hp)
Hd[0] = -.000001
H = Hu @ Hd.diag() @ Hu.t()
print('torch eigh = ', torch.linalg.eigh(H).eigenvalues[0].item())
print('torch eig =', torch.linalg.eig(H).eigenvalues.real.sort().values[0].item())
print('numpy eigh =', np.linalg.eigh(H.numpy()).eigenvalues[0])
print('numpy eig =', sorted(np.linalg.eig(H.numpy()).eigenvalues.real)[0])
print('torch eigh (float64) =', torch.linalg.eigh(H.to(dtype = torch.float64)).eigenvalues[0].item())

order3_ = torch.randn(S).abs()
order3_[1] = 0

lrs, logs = nesterov_lrs(H, g, order3_)
print('lrs =', lrs)
print('logs:', logs)

torch eigh =  -1.4305149989013444e-06
torch eig = -1.225428491125058e-06
numpy eigh = -1.1945735e-06
numpy eig = -1.1770759e-06
torch eigh (float64) = -1.1945735670630374e-06
lrs = tensor([ 0.2405,  0.1780, -0.1238, -0.9125, -0.4407, -0.1025, -0.2035, -0.7663,
        -0.8182,  0.2364])
logs: {'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_converged': True, 'x0': 5.636356074173739e-06, 'f(x0)': 12788049.434618205, 'x1': 3.000016909068222, 'f(x1)': -2.5834052607080373, 'r': 1.0440390629176115, 'r_converged': True, 'found': True}


In [4]:
### Building test

def test_nesterov(S, mode_H = 'random', mode_order3_ = 'random'):
    g = torch.randn(S)
    
    Hm = torch.randn(S, S)
    Hp = Hm.t() @ Hm
    Hd, Hu = torch.linalg.eigh(Hp)
    if mode_H == 'random':
        Hd.normal_()
    elif mode_H == 'small':
        Hd[0] = -1e-6
    H = Hu @ Hd.diag() @ Hu.t()
    
    order3_ = torch.randn(S).abs()
    if mode_order3_ == 'zero':
        order3_[1] = 0.
    elif mode_order3_ == 'small':
        order3_[1] = 2e-5
    
    lrs, logs = nesterov_lrs(H, g, order3_)
    #print('lrs =', lrs)
    print('found: {}'.format(logs['found']))
    if logs['found']:
        print('    lrs =', lrs)
    if True:
        print('    logs =', logs)

In [5]:
for P in [2, 5, 10, 20, 30, 50]:
    print('###################')
    print('##### P = {} #####'.format(P))
    print('###################')
    for i in range(5):
        #print('beginning')
        test_nesterov(P, mode_H = 'random', mode_order3_ = 'zero')
        #print('end')
    print('')

###################
##### P = 2 #####
###################
found: True
    lrs = tensor([ 2.3751, -2.4936])
    logs = {'do_float64': True, 'H_pd': True, 'D_sing': True, 'x1': 1.0, 'f(x1)': -0.2144166754858402, 'r': 0.8066164560068635, 'r_converged': True, 'found': True}
found: False
    logs = {'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_converged': False, 'found': False}
found: True
    lrs = tensor([-0.1587,  3.0118])
    logs = {'do_float64': True, 'H_pd': True, 'D_sing': True, 'x1': 1.0, 'f(x1)': -0.8994761375828038, 'r': 0.13169498555692002, 'r_converged': True, 'found': True}
found: True
    lrs = tensor([-4.2234, -1.9514])
    logs = {'do_float64': True, 'H_pd': True, 'D_sing': True, 'x1': 3.0, 'f(x1)': -1.811253046324844, 'r': 1.4672289000373884, 'r_converged': True, 'found': True}
found: False
    logs = {'do_float64': True, 'H_pd': False, 'D_sing': True, 'x0_converged': False, 'found': False}

###################
##### P = 5 #####
###################
found: False
 